In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionimport pandas as pd 


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

train = pd.read_csv('../input/spaceship-titanic/train.csv', index_col='PassengerId')
test = pd.read_csv('../input/spaceship-titanic/test.csv', index_col='PassengerId')

train.head()

In [ ]:
y=train.Transported
X=train.drop(['Transported'], axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

In [ ]:
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

In [ ]:
my_cols = categorical_cols + numerical_cols
X_t = X_train[my_cols].copy()
X_v = X_val[my_cols].copy()
X_test = test[my_cols].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
numerical_transformer = SimpleImputer(strategy='constant')

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [ ]:
from sklearn.metrics import mean_absolute_error
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [ ]:
my_pipeline.fit(X_train, y_train)

In [ ]:
preds = my_pipeline.predict(X_val)
print(preds)


In [ ]:
score = mean_absolute_error(y_val, preds)
print('MAE:', score)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

In [ ]:
print("Average MAE score (across experiments):")
print(scores.mean())

In [ ]:
test.head()

In [ ]:
test_preds=my_pipeline.predict(test)
rounded= test_preds.round()
data_new1 = rounded.copy()                                
data_new1 = data_new1.astype(bool) 
print(data_new1)
test.index.value_counts()

In [ ]:
output = pd.DataFrame({'PassengerId': test.index,
                       'Transported': data_new1})
output.to_csv('submission6.csv', index=False)